In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [2]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [3]:
df_train_core = df_train[['airline_sentiment', 'text']]
df_train_core_without_nan = df_train_core.drop([3338])

In [1]:
big_list = []

for i in range(len(df_train_core_without_nan)):
    each_list = df_train_core_without_nan.text[i].split()
    big_list = big_list2 + each_list

word_count = Counter(big_list)
word_count_most_common = word_count.most_common()

NameError: name 'df_train_core_without_nan' is not defined

In [5]:
big_list = []

for j in range(len(word_count_most_common)):
    
    total_count = 0 
    positive_count = 0 
    neutral_count = 0 
    negative_count = 0 
    word = word_count_most_common[j][0]
    row = []
    
    for i in range(len(df_train_core_without_nan)):
        list1 = df_train_core_without_nan.text[i].split()
        if word in list1:
            total_count = total_count + 1
        if word in list1 and df_train_core_without_nan.airline_sentiment[i] == 'positive':
            positive_count = positive_count + 1
        if word in list1 and df_train_core_without_nan.airline_sentiment[i] == 'neutral':
            neutral_count = neutral_count + 1
        if word in list1 and df_train_core_without_nan.airline_sentiment[i] == 'negative':
            negative_count = negative_count + 1
    row.append(word)
    row.append(total_count)
    row.append(positive_count)
    row.append(neutral_count)
    row.append(negative_count)
    row.append(positive_count/total_count)
    row.append(neutral_count/total_count)
    row.append(negative_count/total_count)
    
    big_list.append(row)

In [6]:
labels = ['word', 'total_count', 'positive_count', 'neutral_count', 'negative_count', 'pos_prob', 'neu_prob', 'neg_prob']
df_train_analyzed = pd.DataFrame.from_records(big_list, columns = labels)

In [7]:
def biggest(x, y, z):
    
    Max = x
    status = 'positive'
    if y > Max:
        Max = y   
        status = 'neutral'
    if z > Max:
        Max = z
        status = 'negative'
        if y > z:
            Max = y
            status = 'neutral'
    return Max, status

In [8]:
df_train_analyzed_rescaled = df_train_analyzed.copy()

e = 2

df_train_analyzed_rescaled['pos_prob'] = df_train_analyzed_rescaled['pos_prob'].apply(lambda x : pow(x, 1/e + 1/6))
df_train_analyzed_rescaled['neu_prob'] = df_train_analyzed_rescaled['neu_prob'].apply(lambda x : pow(x, 1/(2*e + 1/6)))
df_train_analyzed_rescaled['neg_prob'] = df_train_analyzed_rescaled['neg_prob'].apply(lambda x : pow(x, e + 1/6))

In [9]:
c = 0 

for j in range(len(df_train_core_without_nan)):

    list2 = df_train_core_without_nan.text[j].split()
    list2_to_pd = []
    df_sentence = []

    for i in range(len(list2)):
        word2 = list2[i]
        a = df_train_analyzed_rescaled.loc[df_train_analyzed_rescaled['word'] == word2]
        b = a[['word', 'pos_prob', 'neu_prob', 'neg_prob']].values.tolist()
        list2_to_pd.append(b[0])

    df_sentence = pd.DataFrame(list2_to_pd, columns =['word', 'pos_prob', 'neu_prob', 'neg_prob'] )
    k = biggest(df_sentence['pos_prob'].max(), df_sentence['neu_prob'].max(), df_sentence['neg_prob'].max())
    if k[1] == df_train_core_without_nan.airline_sentiment[j]:
        c = c + 1
        
print('accuracy is ', c/len(df_train_core_without_nan))

accuracy is  0.9742360695026963


In [10]:
df_test_core = df_test['text']
big_list = []

for i in range(len(df_test_core)):
    each_list = df_test_core[i].split()
    big_list = big_list + each_list

word_count = Counter(big_list)
word_count_most_common = word_count.most_common()

In [11]:
index = []

for i in range(824):
    index.append(10911 + i)

labels = ['word', 'total_count', 'positive_count', 'neutral_count', 'negative_count', 'pos_prob', 'neu_prob', 'neg_prob']
big_list = []
s = set(df_train_analyzed['word'])
c = 0 
for i in range(len(word_count_most_common)):
    if word_count_most_common[i][0] not in s:
        list3 = []
        list3.append(word_count_most_common[i][0])
        for j in range(7):
            list3.append(0.0)
        big_list.append(list3)
        
df_big_list = pd.DataFrame(big_list, columns = labels, index = index)

In [12]:
df_train_test_combined = pd.concat([df_train_analyzed_rescaled, df_big_list])

In [13]:
c = 0 
labels = ['tweet_id', 'airline_sentiment']
big_result = []

for j in range(len(df_test_core)):

    list2 = df_test_core[j].split()
    list2_to_pd = []
    df_sentence = []
    result = []   

    for i in range(len(list2)):
        word2 = list2[i]
        a = df_train_test_combined.loc[df_train_test_combined['word'] == word2]
        b = a[['word', 'pos_prob', 'neu_prob', 'neg_prob']].values.tolist()
        list2_to_pd.append(b[0])

    df_sentence = pd.DataFrame(list2_to_pd, columns =['word', 'pos_prob', 'neu_prob', 'neg_prob'] )
    k = biggest(df_sentence['pos_prob'].max(), df_sentence['neu_prob'].max(), df_sentence['neg_prob'].max())
    
    result.append(df_test.tweet_id[j])
    result.append(k[1])

    big_result.append(result)

df_submission = pd.DataFrame(big_result, columns = labels)

In [14]:
df_submission.to_csv('output1.csv', encoding='utf-8', index=False)

In [15]:
df_train_test_combined.head()

,word,total_count,positive_count,neutral_count,negative_count,pos_prob,neu_prob,neg_prob
0,@united,2708.0,362.0,498.0,1848.0,0.261441,0.666039,0.436967
1,to,1382.0,168.0,258.0,956.0,0.245396,0.668447,0.450013
2,the,1056.0,169.0,159.0,728.0,0.294769,0.634828,0.446697
3,I,854.0,106.0,176.0,572.0,0.248828,0.684500,0.419629
4,a,866.0,88.0,142.0,636.0,0.217759,0.647955,0.512313


In [16]:
df_train_core_without_nan.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [17]:
airline_sentiment_numeric = []

for i in range(len(df_train_core_without_nan)):
    if df_train_core_without_nan.airline_sentiment[i] == 'negative':
        airline_sentiment_numeric.append(-1)
    if df_train_core_without_nan.airline_sentiment[i] == 'neutral':
        airline_sentiment_numeric.append(0)
    else:
        airline_sentiment_numeric.append(1)

In [18]:
df_train_core_without_nan_and_numeric = df_train_core_without_nan.copy()

df_train_core_without_nan_and_numeric['airline_sentiment'] = df_train_core_without_nan_and_numeric['airline_sentiment'].apply(
{'negative':-1, 'neutral':0, 'positive': 1}.get)

df_train_core_without_nan_and_numeric.head()

,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,1,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,-1,@VirginAmerica it's really aggressive to blast...
4,-1,@VirginAmerica and it's a really big bad thing...


In [19]:
c = 0 

train_images = []
train_labels = []

for j in range(len(df_train_core_without_nan_and_numeric)):

    list2 = df_train_core_without_nan_and_numeric.text[j].split()
    list2_to_pd = []
    df_sentence = []
    list3 = []

    for i in range(len(list2)):
        word2 = list2[i]
        a = df_train_analyzed_rescaled.loc[df_train_analyzed_rescaled['word'] == word2]
        b = a[['word', 'pos_prob', 'neu_prob', 'neg_prob']].values.tolist()
        list2_to_pd.append(b[0])

    df_sentence = pd.DataFrame(list2_to_pd, columns =['word', 'pos_prob', 'neu_prob', 'neg_prob'] )
    list3.append(df_sentence['pos_prob'].max())
    list3.append(df_sentence['neu_prob'].max())
    list3.append(df_sentence['neg_prob'].max())
    
    train_images.append(list3)
    train_labels.append(df_train_core_without_nan_and_numeric.airline_sentiment[j])

In [39]:
df_train_images = pd.DataFrame(train_images)
train_images_to_array = df_train_images.as_matrix(columns=None).astype(np.float)

df_train_labels = pd.DataFrame(train_labels)
train_labels_to_array = np.array(df_train_labels[0].tolist())

Let's try different classification

In [40]:
df_test_core = pd.DataFrame(df_test_core)

c = 0 

test_images = []

for j in range(len(df_test_core)):

    list2 = df_test_core.text[j].split()
    list2_to_pd = []
    df_sentence = []
    list3 = []

    for i in range(len(list2)):
        word2 = list2[i]
        a = df_train_test_combined.loc[df_train_test_combined['word'] == word2]
        b = a[['word', 'pos_prob', 'neu_prob', 'neg_prob']].values.tolist()
        list2_to_pd.append(b[0])

    df_sentence = pd.DataFrame(list2_to_pd, columns =['word', 'pos_prob', 'neu_prob', 'neg_prob'] )
    list3.append(df_sentence['pos_prob'].max())
    list3.append(df_sentence['neu_prob'].max())
    list3.append(df_sentence['neg_prob'].max())
    
    test_images.append(list3)

In [41]:
test_images = pd.DataFrame(test_images)

test_images_to_array = test_images.as_matrix(columns=None).astype(np.float)

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( train_images_to_array, train_labels_to_array,
                                                    test_size=0.2, random_state=42)

Naive Bayesian

In [43]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
y_pred = clf.fit(X_train, y_train).predict(X_test)

print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))
print("Accuracy is ",  (y_test == y_pred).sum() / X_test.shape[0] )

Number of mislabeled points out of a total 668 points : 10
Accuracy is  0.9850299401197605


In [64]:
col1 = df_test['tweet_id'].tolist()
y_pred = clf.fit(X_train, y_train).predict(test_images_to_array)
labels = ['tweet_id', 'airline_sentiment']
d = {'airline_sentiment': y_pred, 'tweet_id': col1}
df = pd.DataFrame(data=d)
df = df.reindex(columns=labels)
df['airline_sentiment'] = df['airline_sentiment'].apply({-1 : 'negative', 0:'neutral', 1:'positive' }.get)

In [65]:
df.to_csv('outputNB.csv', encoding='utf-8', index=False)

KNN

In [66]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
y_pred = clf.fit(X_train, y_train).predict(X_test)

print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))
print("Accuracy is ",  (y_test == y_pred).sum() / X_test.shape[0] )

Number of mislabeled points out of a total 668 points : 12
Accuracy is  0.9820359281437125


In [67]:
col1 = df_test['tweet_id'].tolist()
y_pred = clf.fit(X_train, y_train).predict(test_images_to_array)
labels = ['tweet_id', 'airline_sentiment']
d = {'airline_sentiment': y_pred, 'tweet_id': col1}
df = pd.DataFrame(data=d)
df = df.reindex(columns=labels)
df['airline_sentiment'] = df['airline_sentiment'].apply({-1 : 'negative', 0:'neutral', 1:'positive' }.get)

In [68]:
df.to_csv('outputKNN.csv', encoding='utf-8', index=False)

SVM

In [69]:
from sklearn import svm

clf = svm.SVC()

y_pred = clf.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))
print("Accuracy is ",  (y_test == y_pred).sum() / X_test.shape[0] )

Number of mislabeled points out of a total 668 points : 12
Accuracy is  0.9820359281437125


In [70]:
col1 = df_test['tweet_id'].tolist()
y_pred = clf.fit(X_train, y_train).predict(test_images_to_array)
labels = ['tweet_id', 'airline_sentiment']
d = {'airline_sentiment': y_pred, 'tweet_id': col1}
df = pd.DataFrame(data=d)
df = df.reindex(columns=labels)
df['airline_sentiment'] = df['airline_sentiment'].apply({-1 : 'negative', 0:'neutral', 1:'positive' }.get)

In [71]:
df.to_csv('outputSVM.csv', encoding='utf-8', index=False)

Decision Tree

In [72]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()

y_pred = clf.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))
print("Accuracy is ",  (y_test == y_pred).sum() / X_test.shape[0] )

Number of mislabeled points out of a total 668 points : 15
Accuracy is  0.9775449101796407


In [73]:
col1 = df_test['tweet_id'].tolist()
y_pred = clf.fit(X_train, y_train).predict(test_images_to_array)
labels = ['tweet_id', 'airline_sentiment']
d = {'airline_sentiment': y_pred, 'tweet_id': col1}
df = pd.DataFrame(data=d)
df = df.reindex(columns=labels)
df['airline_sentiment'] = df['airline_sentiment'].apply({-1 : 'negative', 0:'neutral', 1:'positive' }.get)

In [74]:
df.to_csv('outputDT.csv', encoding='utf-8', index=False)

Random Forrest

In [75]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

y_pred = clf.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))
print("Accuracy is ",  (y_test == y_pred).sum() / X_test.shape[0] )

Number of mislabeled points out of a total 668 points : 18
Accuracy is  0.9730538922155688


In [76]:
col1 = df_test['tweet_id'].tolist()
y_pred = clf.fit(X_train, y_train).predict(test_images_to_array)
labels = ['tweet_id', 'airline_sentiment']
d = {'airline_sentiment': y_pred, 'tweet_id': col1}
df = pd.DataFrame(data=d)
df = df.reindex(columns=labels)
df['airline_sentiment'] = df['airline_sentiment'].apply({-1 : 'negative', 0:'neutral', 1:'positive' }.get)

In [77]:
df.to_csv('outputRF.csv', encoding='utf-8', index=False)

Tensor Flow

In [78]:
import tensorflow as tf

# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
num_input = 3 # MNIST data input (img shape: 28*28)
n_hidden_1 = 2 # 1st layer number of neurons
n_hidden_2 = 1 # 2nd layer number of neurons
num_classes = 3 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

C:\Users\Jason\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [79]:
# y_train to one hot vetor

depth = 3
y_train_one_hot = tf.one_hot(y_train, depth)
y_train_one_hot_np_array = tf.Session().run(y_train_one_hot)

y_test_one_hot = tf.one_hot(y_test, depth)
y_test_one_hot_np_array = tf.Session().run(y_test_one_hot)

In [80]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [81]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [82]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [83]:
def mini_batches(X_train, y_train, batch_size):
    r = np.random.permutation(len(X_train))[:batch_size]
    batch_x = []
    batch_y = []
    
    for i in range(batch_size):
        batch_x.append(X_train[r[i]])
        batch_y.append(y_train[r[i]])
        
    return batch_x, batch_y    

In [84]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mini_batches(X_train, y_train_one_hot_np_array, batch_size)
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
         
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        
        
    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
          sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot_np_array}))

Step 1, Minibatch Loss= 0.2649, Training Accuracy= 0.141
Step 100, Minibatch Loss= 29.8390, Training Accuracy= 0.000
Step 200, Minibatch Loss= 172.8518, Training Accuracy= 0.000
Step 300, Minibatch Loss= 465.0149, Training Accuracy= 0.000
Step 400, Minibatch Loss= 259.9980, Training Accuracy= 0.852
Step 500, Minibatch Loss= 70.0684, Training Accuracy= 0.875
Optimization Finished!
Testing Accuracy: 0.8338323


In [86]:
biases

{'b1': <tf.Variable 'Variable_3:0' shape=(2,) dtype=float32_ref>,
 'b2': <tf.Variable 'Variable_4:0' shape=(1,) dtype=float32_ref>,
 'out': <tf.Variable 'Variable_5:0' shape=(3,) dtype=float32_ref>}

In [ ]:
y = tf.nn.softmax(tf.matmul(x,weights) + biases)

feed_dict = {x: [your_image]}
classification = tf.run(y, feed_dict)
print classification